In [ ]:
!pip install -q sklearn
%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

Load Dataset

In [ ]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

#Classify the input dataset
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

Look at the data

In [ ]:
dftrain.head()
dftrain.describe()
dftrain.shape

y_train(survived) information

In [ ]:
y_train.head()

Graph of the data

In [ ]:
dftrain.age.hist(bins=20)

In [ ]:
dftrain.sex.value_counts().plot(kind = 'barh')

In [ ]:
dftrain['class'].value_counts().plot(kind='barh')

In [ ]:
pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind = 'barh').set_xlabel('% survive')

Training vs Testing Data

In [ ]:
dfeval.shape # testing data set

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []


for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique() # get a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype = tf.float32))



print(feature_columns)


Training Process

Input Function

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df)) # create tf.Dataset object
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs = 1, shuffle = False)
  

Creating the Model with LinearClassifier

In [ ]:
from tensorflow.python.feature_column import feature_column
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns)


Train the model

In [ ]:
linear_est.train(train_input_fn) # train
result = linear_est.evaluate(eval_input_fn) # get model metrics/stats by testing on testing data
clear_output() # clears console output
print(result["accuracy"]) # the result variable is simply a dic of stats about our model
print(result)

Results

In [ ]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[0])
print(y_eval.loc[0])
#probability not surviving
print("probability of not surviving:", result[0]['probabilities'][0])
#probability surviving
print("probability of surviving:", result[0]['probabilities'][1])